In [1]:
from InsurancePremiumPrediction.entity.artifact_entity import DataIngestionArtifact,\
    DataValidationArtifact,DataTransformationArtifact, ModelTrainerArtifact
from InsurancePremiumPrediction.entity.config_entity import ModelTrainerConfig
from InsurancePremiumPrediction.util.util import load_numpy_array_data
import os,sys

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics._regression import r2_score,mean_squared_error
import pickle


In [2]:
os.chdir('B:\Python_projects\Insurance_Prediction')

In [3]:
model_dict={}
linear_model=LinearRegression()
decision_tree_model=DecisionTreeRegressor()
knn_model=KNeighborsRegressor()
gradient_booster=GradientBoostingRegressor()
random_booster=RandomForestRegressor()
model_dict['linear_model']=linear_model
model_dict['decision_tree_model']=decision_tree_model
model_dict['knn_model']=knn_model
model_dict['gradient_booster']=gradient_booster
model_dict['random_booster']=random_booster


        

In [4]:
model_dict

{'linear_model': LinearRegression(),
 'decision_tree_model': DecisionTreeRegressor(),
 'knn_model': KNeighborsRegressor(),
 'gradient_booster': GradientBoostingRegressor(),
 'random_booster': RandomForestRegressor()}

In [5]:
tain_array_path=r'B:\Python_projects\Insurance_Prediction\InsurancePremiumPrediction\artifact\data_transformation\22-07-17-12-03-36\transformed_data\train\insurance.npz'
test_array_path=r'B:\Python_projects\Insurance_Prediction\InsurancePremiumPrediction\artifact\data_transformation\22-07-17-12-03-36\transformed_data\test\insurance.npz'

In [6]:
train_array=load_numpy_array_data(test_array_path)
test_array=load_numpy_array_data(test_array_path)
x_train,y_train,x_vaidation,y_validation=train_array[:,:-1],train_array[:,-1],test_array[:,:-1],test_array[:,-1]

X_train,X_test,Y_train,Y_test=train_test_split(x_train,y_train,random_state=42,test_size=.25)

In [37]:
model_score_set={}
for i in model_dict.values():
    i.fit(X_train,Y_train)
    Y_predict=i.predict(X_test)
    model_score_set[i]=[i.score(X_train,Y_train),r2_score(Y_test,Y_predict)]

In [38]:
model_score_set

{LinearRegression(): [0.7411663050011708, 0.7235116716346139],
 DecisionTreeRegressor(): [1.0, 0.7743290215771234],
 KNeighborsRegressor(): [0.7997540507267018, 0.6693900600920861],
 GradientBoostingRegressor(): [0.9561560087868748, 0.8171190722272956],
 RandomForestRegressor(): [0.9727382430365209, 0.8147036909921247]}

In [9]:
x=list(model_score_set.values())
import numpy as np
x=np.array(x)
max=0
for i in range(5):
    if max <x[i][1]:
        max=x[i][1]

In [10]:
max

0.8181943402944332

In [58]:
para_grid_gradient={
    'learning_rate' : [0.1,0.01],
    'n_estimators': [50,100],
    'criterion':['friedman_mse', 'squared_error'],
    'min_samples_leaf':[2,5,7],
    'max_depth': [None,2,3,4,5],
    'random_state': [None,0,100],
    'alpha': [0.2,0.5,0.7,0.9],
    'ccp_alpha': [0.0,0.01,0.1]
    }


In [59]:
gradient_booster.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [60]:
grid_cv_gradient=GridSearchCV(error_score='raise',estimator=gradient_booster,param_grid=para_grid_gradient,cv=2,n_jobs=-1,verbose=1)

In [61]:
grid_cv_gradient.fit(X_train,Y_train)

Fitting 2 folds for each of 4320 candidates, totalling 8640 fits


GridSearchCV(cv=2, error_score='raise', estimator=GradientBoostingRegressor(),
             n_jobs=-1,
             param_grid={'alpha': [0.2, 0.5, 0.7, 0.9],
                         'ccp_alpha': [0.0, 0.01, 0.1],
                         'criterion': ['friedman_mse', 'squared_error'],
                         'learning_rate': [0.1, 0.01],
                         'max_depth': [None, 2, 3, 4, 5],
                         'min_samples_leaf': [2, 5, 7],
                         'n_estimators': [50, 100],
                         'random_state': [None, 0, 100]},
             verbose=1)

In [64]:
grid_cv_gradient.best_params_

{'alpha': 0.2,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'learning_rate': 0.1,
 'max_depth': 2,
 'min_samples_leaf': 7,
 'n_estimators': 100,
 'random_state': None}

In [65]:
gradient_booster_new=GradientBoostingRegressor(alpha=0.2, max_depth=2, min_samples_leaf=7,n_estimators=100,random_state=None,ccp_alpha=0.0)

In [70]:
gradient_booster_new.fit(X_train,Y_train)
gradient_booster_new.score(X_train,Y_train)

0.8940321722255743

In [67]:
Y_predict_=gradient_booster_new.predict(X_test)

In [68]:
gradient_booster_new.score(X_test,Y_test)

0.8328327757021002

In [69]:
r2_score(Y_test,Y_predict_)

0.8328327757021002

In [71]:
y_predict_validation=gradient_booster_new.predict(x_vaidation)

In [72]:
r2_score(y_validation,y_predict_validation)

0.8759200969403436